In [4]:
import sklearn
import random 

# Pyspark Library #
# SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import os
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [5]:
#Make Spark Session
spark = SparkSession.builder.config( "spark.jars","/root/hadoop/share/hadoop/tools/lib/mysql-connector-java-8.0.27.jar") \
    .master("local").appName("ml_reco").getOrCreate()

In [2]:
#Load tables (buylist, product, category_small, user_recommand)
buylist = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "buylist") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
product = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "product") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
category_small = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "category_small") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
user_recommand = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "user_recommand") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
cart = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "cart") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
search_user = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "search_user") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
like = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "like_product") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [46]:
#특정 비율만큼 추출
buylist = buylist.sample(withReplacement=False,fraction=0.3)
cart = cart.sample(withReplacement=False,fraction=1.0)
search_user = search_user.sample(withReplacement=False,fraction=0.5)
like = like.sample(withReplacement=False,fraction=0.8)

In [47]:
#필요한 값만 남기고 정리 (필요값 : user_id, product_id,
#                           category_(small,mid)_id, avg_star, keyword)
buylist = buylist.drop('buy_date','id','count')
product = product.drop('regist_time','name','price','brand','image')
category_small = category_small.drop('name')
search_user = search_user.drop('id')

In [48]:
product = product.withColumnRenamed('id','product_id')
category_small = category_small.withColumnRenamed('id','category_small_id')

In [49]:
buylist.count()

0

In [50]:
df = cart.unionByName(like,allowMissingColumns=True)
# df = df.unionByName(search_user,allowMissingColumns=True)
df = df.unionByName(buylist,allowMissingColumns=True)

In [51]:
inner_df = df.join(product, on = ["product_id"],how='left').sort("product_id")

In [52]:
inner_df.show()

+----------+-------+-----------------+--------+
|product_id|user_id|category_small_id|avg_star|
+----------+-------+-----------------+--------+
|         1|      1|                1|     4.5|
+----------+-------+-----------------+--------+



In [30]:
aa = spark.range(10)

+---+
| id|
+---+
|  5|
|  6|
|  8|
|  9|
+---+

